In [2]:
import gym
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# Goal: learn right setting of weights to maximize the expected rewards
# Want an NN that outputs distribution over actions
class Agent(nn.Module):
    def __init__(self, n_actions, n_states, hidden_size):
        super(Agent, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(n_states, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size*2),
            nn.ReLU(),
            nn.Linear(hidden_size*2, n_actions),
            nn.Softmax(dim=1) # b/c we want a distribution over action space
        )

    def forward(self, X):
        return self.net(X)

class VPG():
    def __init__(self,n_actions, n_states, hidden_size, buffer_size, learning_rate, batch_size):
        self.agent = Agent(n_actions, n_states, hidden_size)
        self.state_buffer = np.zeros((buffer_size, n_states))
        self.next_state_buffer = np.zeros((buffer_size, n_states))
        self.actions = np.zeros(buffer_size)
        self.rewards = np.zeros(buffer_size)

        self.loss_fn = nn.HuberLoss()
        self.optim = torch.optim.Adam(self.agent.parameters(), lr=learning_rate)
    

In [9]:
n_actions = 4
n_states = 8
hidden_size = 64
buffer_size = 1000000
learning_rate = 0.001
batch_size = 64
model = VPG(n_actions, n_states, hidden_size, buffer_size, learning_rate, batch_size)

env = gym.make("LunarLander-v2", render_mode="human")
observation, info = env.reset(seed=42)
for _ in range(1000):
   action = torch.max(model.agent(observation)).item()
   
   observation, reward, terminated, truncated, info = env.step(action)

   if terminated or truncated:
      observation, info = env.reset()
env.close()

TypeError: linear(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

: 